In [0]:
# ✅ Step 1: Define your Azure Storage info
storage_account_name = "dmodemoaccount123"
storage_account_key = ""  # Replace with actual key from Azure Portal

# ✅ Step 2: Set Spark config for accessing ADLS Gen2
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

In [0]:
processName=dbutils.widgets.get("prm_processName")
print(processName)
# DBTITLE 1

dailyPricingSourceIngest


In [0]:
nextSourceFileDateSql=f"""
SELECT DATE_ADD(MAX(CAST(PROCESS_FILE_TABLE_DATE AS DATE)), 1) as NEXT_SOURCE_FILE_DATE
FROM processrunlogs.deltalakehouse_process_run where PROCESS_STATUS='Completed' and PROCESS_NAME='dailyPricingSourceIngest'
"""

nextSourceFileDateDF=spark.sql(nextSourceFileDateSql)
print(nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE"))
nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE']
print(nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE'])


DataFrame[NEXT_SOURCE_FILE_DATE: date]
2025-03-02


In [0]:
dailyPricingSourceBaseURL="https://retailpricing.blob.core.windows.net/"
dailyPricingSourceFolder="daily-pricing/"
dailyPricingSourceFileName="PW_MW_DR_01012023.csv"

dailyPricingSinkLayerName="bronze"
dailyPricingSinkStorageAccountName=storage_account_name
dailyPricingSinkFolderName="daily-pricing"

In [0]:
import pandas as pds

In [0]:
dailyPricingSourceURL=dailyPricingSourceBaseURL+dailyPricingSourceFolder+dailyPricingSourceFileName
print(dailyPricingSourceURL)

https://retailpricing.blob.core.windows.net/daily-pricing/PW_MW_DR_01012023.csv


In [0]:
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceURL)
dailyPricingPandasDF.head()

,DATE_OF_PRICING,ROW_ID,STATE_NAME,MARKET_NAME,PRODUCTGROUP_NAME,PRODUCT_NAME,VARIETY,ORIGIN,ARRIVAL_IN_TONNES,MINIMUM_PRICE,MAXIMUM_PRICE,MODAL_PRICE
0,01/01/2023,12699,Uttar Pradesh,Najibabad,Fruits,Raddish,Raddish,NR,23,250,350,300.0
1,01/01/2023,12711,Uttar Pradesh,Sahiyapur,Fruits,Raddish,Raddish,NR,2.6,330,415,370.0
2,01/01/2023,12723,Uttar Pradesh,Ujhani,Fruits,Raddish,Other,NR,0.6,450,550,500.0
3,01/01/2023,12735,Uttrakhand,Rudrapur,Fruits,Raddish,Other,NR,2.5,100,300,200.0
4,01/01/2023,12759,Kerala,Manjeswaram,Fruits,Ridge gourd(Tori),Ridgeguard(Tori),NR,5,3400,4400,3900.0


In [0]:
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)
dailyPricingSparkDF.printSchema()

root
 |-- DATE_OF_PRICING: string (nullable = true)
 |-- ROW_ID: long (nullable = true)
 |-- STATE_NAME: string (nullable = true)
 |-- MARKET_NAME: string (nullable = true)
 |-- PRODUCTGROUP_NAME: string (nullable = true)
 |-- PRODUCT_NAME: string (nullable = true)
 |-- VARIETY: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ARRIVAL_IN_TONNES: string (nullable = true)
 |-- MINIMUM_PRICE: string (nullable = true)
 |-- MAXIMUM_PRICE: string (nullable = true)
 |-- MODAL_PRICE: double (nullable = true)



In [0]:
# dbutils.fs.ls("abfss://bronze@dmodemoaccount123.dfs.core.windows.net/daily-pricing")

In [0]:
# dbutils.fs.rm("abfss://bronze@dmodemoaccount123.dfs.core.windows.net/daily-pricing/", recurse=True)

In [0]:
from pyspark.sql.functions import current_timestamp
dailyPricingSynkFolderPath=f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"
print(dailyPricingSynkFolderPath)

(dailyPricingSparkDF
 .withColumn("source_file_load_date",current_timestamp())
 .write
 .mode("append")
 .option("header","true")
 .csv(dailyPricingSynkFolderPath)
)

abfss://bronze@dmodemoaccount123.dfs.core.windows.net/daily-pricing
